## GPT-2を芥川龍之介の作品を用いてファインチューニングする

In [1]:
# Google Driveをマウントと作業用ディレクトリ作成
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
# ファインチューニングした結果を保存するためのディレクトリ作成
!mkdir -p '/content/drive/MyDrive/work/output/'
%cd '/content/drive/My Drive/work/'

Mounted at /content/drive
/content/drive/My Drive/work


In [2]:
# 学習データをダウンロード
!wget https://raw.githubusercontent.com/koit2525/Fake_Akutagawa_sentence/main/train.txt

--2022-06-16 10:08:17--  https://raw.githubusercontent.com/koit2525/Fake_Akutagawa_sentence/main/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1048576 (1.0M) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]   1.00M  --.-KB/s    in 0.03s   

2022-06-16 10:08:17 (38.8 MB/s) - ‘train.txt’ saved [1048576/1048576]



In [3]:
# transformersをインストール
!pip install git+https://github.com/huggingface/transformers
# rinna/japanese-gpt2-mediumのトークナイザーはsentencepieceなのでsentencepieceもインストール
!pip install folium==0.2.1
!pip install sentencepiece
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-nzpq384l
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-nzpq384l
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 86 kB 4.1 MB/s 
     |████████████████████████████████| 596 kB 35.2 MB/s 
     |████████████████████████████████| 6.6 MB 66.2 MB/s 
  Created wheel for transformers: filename=transformers-4.20.0.dev0-py3-none-any.whl size=4437832 sha256=cabbcb0cbdf8e0acb8bd028798237ee3f4816a3d0ce758f3c3162b8d5381da70
  Stored in directory: /tmp/pip-ephem-wheel-cache-vae7fiy1/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninsta

In [4]:
# ファインチューニングにrun_clm.py利用するので、transformaersをダウンロード
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 98470, done.
remote: Counting objects: 100% (666/666), done.
remote: Compressing objects: 100% (343/343), done.
remote: Total 98470 (delta 401), reused 492 (delta 271), pack-reused 97804
Receiving objects: 100% (98470/98470), 91.84 MiB | 11.26 MiB/s, done.
Resolving deltas: 100% (72403/72403), done.
Checking out files: 100% (2296/2296), done.


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/787k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/799 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

In [6]:
# 学習データを用いてファインチューニングを実施
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=/content/drive/MyDrive/work/train.txt \
    --validation_file=/content/drive/MyDrive/work/train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=10 \
    --save_steps=10000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=/content/drive/MyDrive/work/output/ \
    --use_fast_tokenizer=False

06/16/2022 10:10:58 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/16/2022 10:10:58 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=N

## ファインチューニングしたモデルを利用して文章を生成する

In [7]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ファインチューニングしたモデルを読み込む
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/work/output")
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [8]:
# 生成文の文頭となる文章
input_texts = ["ある日の暮方の事である。", "ある日の事でございます。", "僕の将来に対する唯ぼんやりとした不安",]

for text in input_texts:
  input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
  out = model.generate(input_ids, do_sample=True, min_length=100, max_length=400, pad_token_id=tokenizer.eos_token_id)
  sent = tokenizer.batch_decode(out)[0]
  sent = sent.replace('</s>', '')
  sent = sent.replace('<unk>', '')
  sent = sent.replace('。', '。\n')
  print(f"-------「{text}」から生成した芥川龍之介風文章-------")
  print(f"{sent.strip()}\n")

-------「ある日の暮方の事である。」から生成した芥川龍之介風文章-------
ある日の暮方の事である。
 と云う風な事を言つた。
わたしはその時、耳をつんざくばかりの、可笑しいと思つた。
あの調子だから、一々のを鳴らしながら、わたしの言ひつけを聞かないかも知れない。
さう思いながら、わたしは口をとざした。
いつか、あの悪夢から醒めたように、わたしはもう一度この調子で、わたしの言ひつけを言ふ真似事をしたいと思つたからだ。

-------「ある日の事でございます。」から生成した芥川龍之介風文章-------
ある日の事でございます。
 」と云つた。
云つた後も、何度もかう考へた。
「兎に角、信輔は、どうして、今だ信輔の決心は、弟には通じないのであろう? 何にしても、彼の決心は、弟には、容赦がないのに相違ない。
それに、どうして、こんなにも、信輔を憎む気になるのでございましょう。
」信輔は、一瞬間、口角に、髭の生えた、可笑しさを含んだ微笑を感じた。
が、同時に又、複雑な心もちが、込み上げて来た。
それは、疑うのを誤って、自分を殺さなかつた弟に対する、彼の復讐心を、復讐しようとする彼の心が、知らず知らず、そうして、その時は、燃えている、熱した、あの黒煙と同じやうな不快な、そんな忌まわしい、忌まわしい、考えに変らざるを得なかったからである。

-------「僕の将来に対する唯ぼんやりとした不安」から生成した芥川龍之介風文章-------
僕の将来に対する唯ぼんやりとした不安或はこの不安にいつも脅かされ、絶えず怯えながら、しかもそれと共にそれと同時に愉快な悦びを感じない訣には行かない僕の心の問題である。
けれども将来に対する不安は僕を更に苦しめずには措かない。
僕の僕であるためにこの不相変将来に対する不安ばかり生じているのである。
僕の将来に対する不安は「どうにでもなれ」と云う力の根源に違いない。
「どうにでもなれ」と云う力の源に違いないのは、単に僕が「どうにでもなれ」と云う勇気を持つてゐない為だ。
それを「どうにでもなれ。
さうしてしまへば」と云ふ力の源にすれば、僕は力の無い人間になり兼ねない。
「どうにでもなれ力の源になれば」と云ふ勇気は勿論この「どうにでもなれ力の源になれば」と云ふ力を生む為に必要である。
けれどもそれは同時に「どうにもならざる力の源になれば」